Για να εκτελέσετε τα παρακάτω τετράδια, αν δεν το έχετε κάνει ήδη, πρέπει να ορίσετε το κλειδί openai μέσα στο αρχείο .env ως `OPENAI_API_KEY`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )

model = 'text-embedding-ada-002'

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

Στη συνέχεια, θα φορτώσουμε τον Δείκτη Ενσωμάτωσης σε ένα Pandas Dataframe. Ο Δείκτης Ενσωμάτωσης αποθηκεύεται σε ένα αρχείο JSON που ονομάζεται `embedding_index_3m.json`. Ο Δείκτης Ενσωμάτωσης περιέχει τις Ενσωματώσεις για κάθε μία από τις απομαγνητοφωνήσεις του YouTube μέχρι τα τέλη Οκτωβρίου 2023.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

Στη συνέχεια, θα δημιουργήσουμε μια συνάρτηση που ονομάζεται `get_videos` η οποία θα αναζητά στον Ενσωματωμένο Δείκτη (Embedding Index) για το ερώτημα. Η συνάρτηση θα επιστρέφει τα 5 κορυφαία βίντεο που είναι πιο παρόμοια με το ερώτημα. Η συνάρτηση λειτουργεί ως εξής:

1. Πρώτα, δημιουργείται ένα αντίγραφο του Ενσωματωμένου Δείκτη.
2. Στη συνέχεια, υπολογίζεται η Ενσωμάτωση (Embedding) για το ερώτημα χρησιμοποιώντας το OpenAI Embedding API.
3. Έπειτα, δημιουργείται μια νέα στήλη στον Ενσωματωμένο Δείκτη που ονομάζεται `similarity`. Η στήλη `similarity` περιέχει την συνημίτονα ομοιότητα (cosine similarity) μεταξύ της Ενσωμάτωσης του ερωτήματος και της Ενσωμάτωσης για κάθε τμήμα βίντεο.
4. Στη συνέχεια, ο Ενσωματωμένος Δείκτης φιλτράρεται με βάση τη στήλη `similarity`. Ο Ενσωματωμένος Δείκτης φιλτράρεται ώστε να περιλαμβάνει μόνο βίντεο που έχουν συνημίτονα ομοιότητα μεγαλύτερη ή ίση με 0.75.
5. Τέλος, ο Ενσωματωμένος Δείκτης ταξινομείται με βάση τη στήλη `similarity` και επιστρέφονται τα 5 κορυφαία βίντεο.


In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Αυτή η συνάρτηση είναι πολύ απλή, απλώς εκτυπώνει τα αποτελέσματα του ερωτήματος αναζήτησης.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Πρώτα, ο Δείκτης Ενσωμάτωσης φορτώνεται σε ένα Pandas Dataframe.
2. Στη συνέχεια, ζητείται από τον χρήστη να εισάγει ένα ερώτημα.
3. Έπειτα, καλείται η συνάρτηση `get_videos` για να αναζητήσει τον Δείκτη Ενσωμάτωσης για το ερώτημα.
4. Τέλος, καλείται η συνάρτηση `display_results` για να εμφανίσει τα αποτελέσματα στον χρήστη.
5. Στη συνέχεια, ζητείται από τον χρήστη να εισάγει ένα άλλο ερώτημα. Αυτή η διαδικασία συνεχίζεται μέχρι ο χρήστης να εισάγει `exit`.

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc.el.png)

Θα σας ζητηθεί να εισάγετε ένα ερώτημα. Εισάγετε ένα ερώτημα και πατήστε enter. Η εφαρμογή θα επιστρέψει μια λίστα βίντεο που σχετίζονται με το ερώτημα. Η εφαρμογή θα επιστρέψει επίσης έναν σύνδεσμο στο σημείο του βίντεο όπου βρίσκεται η απάντηση στην ερώτηση.

Εδώ είναι μερικά ερωτήματα για να δοκιμάσετε:

- Τι είναι το Azure Machine Learning;
- Πώς λειτουργούν τα συνελικτικά νευρωνικά δίκτυα;
- Τι είναι ένα νευρωνικό δίκτυο;
- Μπορώ να χρησιμοποιήσω τα Jupyter Notebooks με το Azure Machine Learning;
- Τι είναι το ONNX;


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from input
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Αποποίηση ευθυνών**:  
Αυτό το έγγραφο έχει μεταφραστεί χρησιμοποιώντας την υπηρεσία αυτόματης μετάφρασης AI [Co-op Translator](https://github.com/Azure/co-op-translator). Παρόλο που επιδιώκουμε την ακρίβεια, παρακαλούμε να λάβετε υπόψη ότι οι αυτόματες μεταφράσεις ενδέχεται να περιέχουν λάθη ή ανακρίβειες. Το πρωτότυπο έγγραφο στη μητρική του γλώσσα πρέπει να θεωρείται η αυθεντική πηγή. Για κρίσιμες πληροφορίες, συνιστάται επαγγελματική ανθρώπινη μετάφραση. Δεν φέρουμε ευθύνη για τυχόν παρεξηγήσεις ή λανθασμένες ερμηνείες που προκύπτουν από τη χρήση αυτής της μετάφρασης.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
